Importing libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import re
from json.decoder import JSONDecodeError
import datetime

Reading CSV Files

In [2]:
import import_ipynb
import funciones 

class Director:
    def __init__(self):
        self.films = []
        self.nombre = ""
        self.retorno_total = 0.0

    def add_films(self, film):
        self.films.append(film)

    def get_films(self):
        return self.films
    
    def set_nombre(self, value):
        self.nombre = value
    
    def add_retorno(self, value):
        self.retorno_total = self.retorno_total + float(value)

    def get_nombre(self):
        return self.nombre
    
    def to_string_info_director(self):
        message=f"El Director/Directora: {self.get_nombre()} Tuvo un retorno de: {self.get_retorno_total()} Información de sus peliculas a continuación "

        message2 =  [''.join(film.to_string_info_by_director()) for film in self.films]
        message = f"{message} {str(message2)}"
        return message

    def get_retorno_total(self):
        return self.retorno_total

class Actor:
    def __init__(self):
        self.no_films=0
        self.name=""
        self.retorno=0
        self.promedio=0
    
    def set_no_films(self, value):
        self.no_films = value

    def set_name(self, value):
        self.name = value

    def set_retorno(self, value):
        self.retorno = value

    def set_promedio(self, value):
        self.promedio = value

    def get_no_films(self):
        return self.no_films

    def get_name(self):
        return self.name

    def get_retorno(self):
        return self.retorno

    def get_promedio(self):
        return self.promedio

    def to_string_actor_info(self):
        return "El actor "+self.name+" ha participado de "+str(self.no_films)+" cantidad de filmaciones, el mismo ha conseguido un retorno de "+ str(self.retorno)+ " con un promedio de "+str(self.promedio)+ " por filmación"

class Info_Film:
    def __init__(self):
        self.title = ""
        self.year_movie = ""
        self.popularity = ""
        self.valoraciones = ""
        self.promedio = ""
        self.retorno = 0
        self.fecha_lanzamiento = ""
        self.costo = ""
        self.ganancia = 0

    def set_title(self, value):
        self.title = value
    
    def set_year_movie(self, value):
        self.year_movie = value
    
    def set_popularity(self, value):
        self.popularity = value

    def set_valoraciones(self, value):
        self.valoraciones = value
    
    def set_promedio(self, value):
        self.promedio = value
    
    def set_retorno(self, value):
        self.retorno = value

    def set_fecha_lanzamiento(self, value):
        self.fecha_lanzamiento = value
    
    def set_costo(self, value):
        self.costo = value
    
    def set_ganancia(self, value):
        self.ganancia = value
    
    def get_title(self):
        return self.title
    
    def get_year_movie(self):
        return self.year_movie
    
    def get_popularity(self):
        return self.popularity
    
    def get_valoraciones(self):
        return self.valoraciones
    
    def get_promedio(self):
        return self.promedio
    
    def get_retorno(self):
        return self.retorno
    
    def get_fecha_lanzamiento(self):
        return self.fecha_lanzamiento
    
    def get_costo(self):
        return self.costo
    
    def get_ganancia(self):
        return self.ganancia
    
    def to_string(self):
        return "La película "+self.title+" fue estrenada en el año "+self.year_movie+" con un score/popularidad de "+self.popularity
    
    def to_string_votos_titulo(self):
        message = "La película "+self.get_title()+" fue estrenada en el año "+self.get_year_movie()
        message = message +" La misma cuenta con un total de "+str(self.get_valoraciones())
        if int(self.get_valoraciones()) == 0:
            message = message + " valoraciones, debido a que su popularidad no cumplio con la condición de al menos un valor de 2000"
        else:
            message = message + " valoraciones"
        message = message + " con un promedio de "+self.get_promedio()

        return message
    def to_string_info_by_director(self):
        message = "La película "+self.get_title()+" Su  fecha de lanzamiento es: "+self.get_fecha_lanzamiento()+" Dío un retorno de: "+str(self.get_retorno())
        message = message +" Su costo fue: "+str(self.get_costo())+" y ganancia de "+str(self.get_ganancia())
        return message

    
class Films:
    
    def __init__(self):
        self.movies_final = self.init_movies()
        self.credits_final = self.init_credits()

    def get_credits_final(self):
        return self.credits_final
    
    def get_movies_final(self):
        return self.movies_final

    def init_movies(self):
        movies_dataset = pd.read_csv('resources/movies_dataset.csv',  encoding='utf8')
        movies_df_english_idx = movies_dataset[movies_dataset["spoken_languages"].str.contains("English|Español") == False].index
        movies_pre = movies_dataset.drop(movies_df_english_idx)
        movies_pre.drop_duplicates(inplace=True)
        movies_final = funciones.format_dataset_movies(movies_pre)
        return movies_final
        
    def init_credits(self):
        credits_dataset = pd.read_csv('resources/credits.csv',  encoding='latin-1')
        credits_df_idx = credits_dataset[credits_dataset["id"].isnull()].index
        credits_pre = credits_dataset.drop(credits_df_idx)
        credits_pre.drop_duplicates(inplace=True)
        credits_final = funciones.format_dataset_credits(credits_pre)
        return credits_final

    def get_actor_by_credits(self, credits, target_name, search_id_result = []):
        target_key = "name"
        count = 0

        for index, row in credits.iterrows():
            search_name_result = []
            flag = False
            funciones.json_full_search(target_key, row['cast_json'], search_name_result)
            for name in search_name_result:
                search_director_name = []
                if(name == target_name):
                    funciones.json_job_search("job", "Director", target_name, row['crew_json'], search_director_name)
                    if len(search_director_name) > 0:
                        for director in search_director_name:
                            if (director == target_name):
                                flag = False;
                            else:
                                flag = True
                    else:
                        flag = True
            if flag == True:
                count = count + 1
                search_id_result.append(row['id'])

        return count
    
    
    def get_actor(self, actor_name):
        search_id_result = []
        actor = Actor()
        actor.set_no_films(self.get_actor_by_credits(self.credits_final, actor_name, search_id_result))
        actor.set_name(actor_name)
        actor_return = []
        for id in search_id_result:
            id_int = int(id)
            
            row = self.movies_final[self.movies_final['id'] == id_int]
            actor_return.append(row['retorno'].values[0])

        my_actor_value = pd.Series(actor_return)
        print(my_actor_value)
        actor.set_retorno(my_actor_value.sum())
        actor.set_promedio(my_actor_value.mean())
        return actor
    
    def cantidad_filmaciones_mes(self, month):
        numeric_month=funciones.get_month_numeric(month)
        str_var = self.movies_final.groupby(['month_movies'])['month_movies'].count()[numeric_month]
        return str_var
        
    
   
    def cantidad_filmaciones_dia(self, day):
        day_formatted = day.lower()
        print(day_formatted)
        return self.movies_final.groupby(['day_movies'])['day_movies'].count()[day_formatted]
        

    def score_titulo(self, film_title):
        title = film_title.lower()
        
        info = Info_Film()
        info.set_title(film_title)
        info.set_popularity(str(self.movies_final[self.movies_final['key_title'] == title]['popularity'].values[0]))
        info.set_year_movie(str(self.movies_final[self.movies_final['key_title'] == title]['year_movies'].values[0]))
        info.set_fecha_lanzamiento(str(self.movies_final[self.movies_final['key_title'] == title]['release_date'].values[0]))
        info.set_retorno(str(self.movies_final[self.movies_final['key_title'] == title]['retorno'].values[0]))
        info.set_costo(str(self.movies_final[self.movies_final['key_title'] == title]['budget'].values[0]))
        info.set_ganancia(str(self.movies_final[self.movies_final['key_title'] == title]['revenue'].values[0]))
        
        return info
        
    def votos_titulo(self, film_title ):
        title = film_title.lower()
        
        info = Info_Film()
        info.set_title(film_title)
        valoraciones = str(self.movies_final[self.movies_final['key_title'] == title]['vote_count'].values[0])
        if int(valoraciones) >= 2000:
            info.set_valoraciones(valoraciones)
        else:
            info.set_valoraciones(0)
        info.set_promedio(str(self.movies_final[self.movies_final['key_title'] == title]['vote_average'].values[0]))
        info.set_year_movie(str(self.movies_final[self.movies_final['key_title'] == title]['year_movies'].values[0]))
        

        return info
    
    def get_director_info(self, movies, credits, target_name, search_id_result = []):
        target_key = "name"
        count = 0
        films_owner_director = []
        for index, row in credits.iterrows():
            flag = False
            search_director_name = []
            funciones.json_job_search("job", "Director", target_name, row['crew_json'], search_director_name)
            for name in search_director_name:
                if name == target_name:
                    films_owner_director.append(row['id'])
            
        films_id_director = [int(id) for id in films_owner_director]
        print("los ids son: ",films_id_director)
        movies_df_title = movies[movies['id'].isin(films_id_director)] 
        list_movies_title = movies_df_title['title']
        director = Director()
        director.set_nombre(target_name)
        for movie_title in list_movies_title.tolist():
            print("Movie is:", movie_title)
            film = self.score_titulo(str(movie_title))
            director.add_retorno(film.get_retorno())
            director.add_films(film)
        return director
             
        #Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
        # Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

    def get_director(self, director_name):
        director = self.get_director_info(self.movies_final, self.credits_final, director_name, search_id_result = [])
        
       
        return director

